In [1]:
# Jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
import ray
ray.shutdown()

In [3]:
import ray.rllib
import ray.tune 
import solara.envs.creator

## Initialising ray (starts background process for distributed computing)
ray.shutdown()
ray.init(logging_level="WARNING", object_store_memory= 25 * 10**9)

# Adding environment creator function to ray
ray.tune.registry.register_env("battery_control", solara.envs.creator.create_env)

# Output format of figures
OUT_FORMAT = ".svg"

In [4]:
from solara.constants import PROJECT_PATH
import solara.utils.rllib

EXPERIMENT_NAME = "experiment_08_test_beogym"

# RL environment configuration
ENV_CONFIG = {
    'general': {
        'obs_keys': ["load", "pv_gen", "battery_cont", "time_step"],
        'type': 'battery_control.BatteryControlEnv',
        'infeasible_control_penalty': ray.tune.grid_search([False, True]),
        'grid_charging': ray.tune.grid_search([True, False]),
        'logging_level': "RAY", # if using RLlib, set to 'RAY'
    },
    'components': {
        'battery': {
            'type': 'LithiumIonBattery',
            'size': 10,
            'chemistry': 'NMC',
            'time_step_len': 1,
        },
        'solar': {
            'type': 'DataPV',
            'data_path': PROJECT_PATH + "/data/ausgrid/processed/house2_solar_gen.txt",
            'fixed_sample_num': 200,
        },
        'load': {
            'type': 'DataLoad',
            'data_path': PROJECT_PATH + "/data/ausgrid/processed/house2_combined_load.txt",
            'fixed_sample_num': 200,
        },
        'grid': {
            'type': 'PeakGrid',
            'peak_threshold': 1.0,
        },
    },
}

# RL agent configuration
AGENT_CONFIG = {
    "env": "battery_control",
    "env_config": ENV_CONFIG,
    "gamma": 0.9999999,
    "lr": 5e-5,
    "model": {
        "fcnet_hiddens": [256, 256, 256, 256],
        "fcnet_activation": "relu",
        "post_fcnet_activation": "tanh",
    },
    # Utilities settings
    "framework": "torch",
    "log_level": "WARNING",
    #"num_workers": 9,
    #"num_gpus": 1,
    "callbacks": solara.utils.rllib.InfoCallback,
    "seed" : ray.tune.randint(0, 10000000),
}

# Full experiment configuration including RL algorithm type
EXPERIMENT_CONFIG = {
    "run_or_experiment": "PPO",
    "config": AGENT_CONFIG,
    "stop": {"training_iteration": 100},
    "name": EXPERIMENT_NAME,
    "local_dir": "./tmp/tune/",
    "log_to_file": True,
    "checkpoint_freq": 1,
}

# Other settings
PLOT_DIR = PROJECT_PATH + "/figures/experiments/"

In [5]:
# Parallelisation Setup
if False:
    num_workers = 4
    gpu_count = 1
    reserved_capacity = 0.01 # Driver GPU
    num_gpus_per_worker = (gpu_count - reserved_capacity) / num_workers


    AGENT_CONFIG["num_workers"] = num_workers
    AGENT_CONFIG["num_gpus"] = num_gpus
    AGENT_CONFIG["num_envs_per_worker"]= 8
    

#AGENT_CONFIG["num_gpus"] = 1
#AGENT_CONFIG["num_envs_per_worker"]= 8
AGENT_CONFIG["num_workers"] = 10
AGENT_CONFIG["num_gpus"] = 1
#AGENT_CONFIG["remote_worker_envs"]= True

In [6]:
# Setting visualisation in notebook
reporter = ray.tune.JupyterNotebookReporter(overwrite=True)
reporter.add_metric_column("custom_metrics/cost_mean")
reporter.add_metric_column("custom_metrics/power_diff_mean")

# Running experiment
analysis = ray.tune.run(
    progress_reporter=reporter,
    **EXPERIMENT_CONFIG,
    #resume=True,
)

Trial name,status,loc,env_config/general/grid_charging,env_config/general/infeasible_control_penalty,seed,iter,total time (s),ts,reward,custom_metrics/cost_mean
PPO_battery_control_bc693_00000,RUNNING,192.168.34.159:159171,True,False,6893354,4,41.7865,16000,-21.3653,21.3653
PPO_battery_control_bc693_00001,PENDING,,False,False,4848991,,,,,
PPO_battery_control_bc693_00002,PENDING,,True,True,3656305,,,,,
PPO_battery_control_bc693_00003,PENDING,,False,True,8410283,,,,,


2021-11-05 19:40:29,917	ERROR tune.py:545 -- Trials did not complete: [PPO_battery_control_bc693_00000, PPO_battery_control_bc693_00001, PPO_battery_control_bc693_00002, PPO_battery_control_bc693_00003]
2021-11-05 19:40:29,918	WARNING tune.py:554 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
